In [ ]:
!apt-get update && apt-get install -y wget ffmpeg
!pip install pandas

In [ ]:
!ls -lh /experiment_dir/dataset/

In [ ]:
%set_env DATASET_DIR=/experiment_dir/dataset
%set_env PRETRAINED_MODELS_DIR =/experiment_dir/pretrained_models

In [ ]:
# Download Oxford Town Center Dataset video file and csv annotation file
!bash ./download_towncenter_video_and_labels.sh

In [ ]:
# Extract images from video, resize them and save in images directory with `ffmpeg`
!mkdir $DATASET_DIR/images
!ffmpeg -i $DATASET_DIR/TownCentreXVID.avi -q:v 1 -start_number 0 -frames:v 4501 $DATASET_DIR/images/%d.jpg

In [ ]:
# Create xml annotation file and save in xmls directory
!python create_xmls.py --annotation_path $DATASET_DIR/TownCentre-groundtruth.top

In [ ]:
!python create_tfrecord.py --data_dir $DATASET_DIR --output_dir $DATASET_DIR --label_map_path ./label_map.pbtxt

In [ ]:
!mkdir $PRETRAINED_MODELS_DIR

In [ ]:
!ls  -lh $PRETRAINED_MODELS_DIR

In [ ]:
!wget http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_coco_2018_03_29.tar.gz -O $PRETRAINED_MODELS_DIR/ssd_mobilenet_v2_coco_2018_03_29.tar.gz
!wget http://download.tensorflow.org/models/object_detection/ssdlite_mobilenet_v2_coco_2018_05_09.tar.gz -O $PRETRAINED_MODELS_DIR/ssdlite_mobilenet_v2_coco_2018_05_09.tar.gz
!wget http://download.tensorflow.org/models/object_detection/faster_rcnn_resnet50_coco_2018_01_28.tar.gz -O $PRETRAINED_MODELS_DIR/faster_rcnn_resnet50_coco_2018_01_28.tar.gz

In [ ]:
!tar -xzvf $PRETRAINED_MODELS_DIR/ssd_mobilenet_v2_coco_2018_03_29.tar.gz -C $PRETRAINED_MODELS_DIR

In [ ]:
!mkdir /experiment_dir/ped_ssd_mobilenet_v2
%set_env PIPELINE_CONFIG_PATH=/repo/training/tf_object_detection_api/pedestrian_detection/configs/ssd_mobilenet_v2_pedestrian.config
%set_env MODEL_DIR=/experiment_dir/ped_ssd_mobilenet_v2
%set_env NUM_TRAIN_STEPS=50000
%set_env SAMPLE_1_OF_N_EVAL_EXAMPLES=1

In [ ]:
!python /models/research/object_detection/model_main.py --pipeline_config_path=${PIPELINE_CONFIG_PATH} --model_dir=${MODEL_DIR} --num_train_steps=${NUM_TRAIN_STEPS} --sample_1_of_n_eval_examples=$SAMPLE_1_OF_N_EVAL_EXAMPLES --alsologtostderr

In [ ]:
!ls $MODEL_DIR


In [ ]:
!mkdir -p $MODEL_DIR/frozen_graph
%set_env INPUT_TYPE=image_tensor
%set_env TRAINED_CKPT_PREFIX=/experiment_dir/ped_ssd_mobilenet_v2/model.ckpt-33387
%set_env EXPORT_DIR=/experiment_dir/ped_ssd_mobilenet_v2/frozen_graph

In [ ]:
!python /models/research/object_detection/export_inference_graph.py \
    --input_type=${INPUT_TYPE} \
    --pipeline_config_path=${PIPELINE_CONFIG_PATH} \
    --trained_checkpoint_prefix=${TRAINED_CKPT_PREFIX} \
    --output_directory=${EXPORT_DIR}

In [ ]:
!ls $MODEL_DIR

In [ ]:
!mkdir -p $MODEL_DIR/frozen_graph_tflite
%set_env OUTPUT_DIR=/experiment_dir/ped_ssd_mobilenet_v2/frozen_graph_tflite

In [ ]:
!python /models/research/object_detection/export_tflite_ssd_graph.py \
--pipeline_config_path=$PIPELINE_CONFIG_PATH \
--trained_checkpoint_prefix=$TRAINED_CKPT_PREFIX \
--output_directory=$OUTPUT_DIR \
--add_postprocessing_op=true \
--max_detections=50

In [ ]:
!ls $MODEL_DIR/frozen_graph_tflite